# cr3 CTF 2024
##   guessing-revenge  | 417 pts

Task description:

```
I am very disappointed, please try this one :sob:

$ nc 1337.sb 20004

Author: kibastian, es3n1n
```

Attachments: [main.sage](./main.sage)

The most interesting part is this function:

In [44]:
def submit_b(B):
    if not B.is_invertible():
        return "non invertible"

    n = B.nrows()
    if n != FLAG_N:
        return "rows"

    diagonal_element = B[0, 0]

    identity_multiple = Matrix(F, identity_matrix(n))
    for i in range(n):
        identity_multiple[i, i] = diagonal_element

    if B == identity_multiple:
        return "identity_multiple"

    return B * A * (B ^ -1) * (A ^ -1) * FLAG


print("aight, lets see what you got")
while True:
    try:
        matrix = deserialize_mtx(input())
        res = submit_b(matrix)
        if isinstance(res, str):
            print("https://youtu.be/G7b4VMEATNo?t=18")
            break
    except Exception as e:
        print("https://youtu.be/G7b4VMEATNo?t=19")
        break

    print(serialize_mtx(res))

## Solution

They fixed the bug and now identity matrix check is consistent.

We need to somehow recover flag from the equation:

$B * A * B^{-1} * A^{-1} * \vec{flag}$, where $B$ should be non identity. 

First guess is permutations.

Let's look at permutations of order 3 on 3x3 matrices:

\begin{align*}
A &= \begin{pmatrix}
a_{00} & a_{01} & a_{02} \\
a_{10} & a_{11} & a_{12} \\
a_{20} & a_{21} & a_{22} \\
\end{pmatrix}
\end{align*}

\begin{align*}
A^{-1} &= \begin{pmatrix}
b_{00} & b_{01} & b_{02} \\
b_{10} & b_{11} & b_{12} \\
b_{20} & b_{21} & b_{22} \\
\end{pmatrix}, \\
\end{align*}

We have 6 permutations on 3 elements, namely:

(1, 2, 3)

(1, 3, 2)

(2, 1, 3)

(2, 3, 1)

(3, 1, 2)

(3, 2, 1)

First one is an identity so we don't need it.

We're left with 5 of them, and let's forget for a moment about the leftmost matrix, since we can easily erase it in the end by multiplying with it's inverse.

We know that right permutation matrix permutes the columns of matrix $A$

Let's take a look at 00 element of the resulting product($A * P * A^{-1}$) using each of 5 permutations, noting that we need to somehow obtain $a_{00} * b_{00} + a_{01} * b_{10} + a_{02} * b_{20} = 1$:

(1, 3, 2):

$r_1 = a_{00}*b_{00} + a_{02}*b_{10} + a_{01}*b_{20}$

(2, 1, 3)

$r_2 = a_{01}*b_{00} + a_{00}*b_{10} + a_{02}*b_{20}$

(2, 3, 1)

$r_3 = a_{02}*b_{00} + a_{00}*b_{10} + a_{01}*b_{20}$

(3, 1, 2)

$r_4 = a_{01}*b_{00} + a_{02}*b_{10} + a_{00}*b_{20}$

(3, 2, 1)

$r_5 = a_{02}*b_{00} + a_{01}*b_{10} + a_{00}*b_{20}$

Now look at $r_1 + r_2 + r_5 - r_3 - r_4$ (It looked better on paper...)

\begin{aligned}
& (a_{00} \cdot b_{00} + a_{02} \cdot b_{10} + a_{01} \cdot b_{20}) + \\
& (a_{01} \cdot b_{00} + a_{00} \cdot b_{10} + a_{02} \cdot b_{20}) + \\
& (a_{02} \cdot b_{00} + a_{01} \cdot b_{10} + a_{00} \cdot b_{20}) - \\
& (a_{02} \cdot b_{00} + a_{00} \cdot b_{10} + a_{01} \cdot b_{20}) - \\
& (a_{01} \cdot b_{00} + a_{02} \cdot b_{10} + a_{00} \cdot b_{20}) \\
&= a_{00} \cdot b_{00} + a_{02} \cdot b_{20} + a_{01} \cdot b_{10} \\
&= 1
\end{aligned}

And we end up with identity.

As simple as that we can use these 3x3 matrices inside the 55x55 matrix.

In [4]:
p = (2 ** 31) - 1
F = GF(p)
FLAG_N = 55

def serialize_mtx(mtx):
    rows = []
    for row in mtx.rows():
        rows.append(','.join(str(elem) for elem in row))
    return '[' + ';'.join(rows) + ']'


def deserialize_mtx(mtx):
    print(mtx)
    rows = mtx.strip('[]\n').split(';')
    rows = [[int(num) for num in row.split(',')] for row in rows]
    return Matrix(F, rows)

In [7]:
from itertools import permutations


def get_permutations():
    perms = []
    for x in permutations([[1, 0, 0], [0, 1, 0], [0, 0, 1]]):
        perms.append(Matrix(F, x))
    return perms[1:]
perms = get_permutations()

In [9]:
from pwn import remote, context
context.log_level = "error"

host, port = "1337.sb",  20004
r = remote(host,port)

# handle pow
print(r.recvline()) 
print(r.recvline())
r.sendline(input().encode())

print(r.recvline())
mtx = identity_matrix(F, FLAG_N)
rets = []
for perm in perms:
    mtx.set_block(0, 0, perm)
    ser = serialize_mtx(mtx)
    r.sendline(ser.encode())
    ret_ = r.recvline()
    ret = deserialize_mtx(ret_.decode())
    rets.append(mtx.inverse() * ret)

print(bytes(list((rets[0] + rets[1] + rets[4] - rets[2] - rets[3]).T)[0]))

b'proof of work:\n'
b'curl -sSfL https://pwn.red/pow | sh -s s.AAATiA==.ybgCkbpqvNFCeVE9vG08Mg==\n'


 s.E/pF80wr6gAJQ3MwyIOE5JoKMJAFAYDoOjkkIMjcctvULH5Gjjcyprhom4TiAQybYKGVd153zJz+tnKT+CKepKAFJVeiGcNWP94vo74AJQnEKZcBicWyWWNzEhPyE87HHcB6HQPKQaiUVp06LJbOvShk0/BCc/w1x0mgTDj1+7M2B9frUYVTTaveorcjBvHZaWvkNm3ZAIGmBCJdoq8U6g==


b'solution: aight, lets see what you got\n'
[486079067;88420838;1600071321;231949131;258963969;2100659830;1854190262;1101715659;152973739;1947761011;308018709;277556764;1403695992;114689910;197397507;1793399019;1703520027;1289110046;649239307;779818236;1239475280;310870980;258093726;1035403571;231405879;463995621;845290517;905842707;1990093616;252898826;602290120;537778130;1943944997;503464440;796708699;200077565;779418537;1875001329;130908731;1810109982;22523135;525387473;2057922684;420460398;462089708;1792594215;1217892639;1939879344;1841690155;226042940;503166034;1294317534;976433285;1777915604;607031138]

[642107504;1698303104;1882968274;1102571296;1941408535;828200353;375005863;1449498279;1420652419;2127299918;1513120052;1318701630;449450586;602903819;11569380;1637682333;39797519;1254771925;55723384;853368808;641589075;759438142;507365641;684407578;1984300941;2112513047;2006527161;1079159296;658853623;994318879;1788382229;707531584;443046558;684099138;1861779469;134266539;51843542